In [ ]:
import mne
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.callbacks import ModelCheckpoint, EarlyStopping
from EEGNet import EEGNet
from CustomNet import CustomNet
mne.set_log_level(40)

In [ ]:
epochs = mne.read_epochs('unprocessed/epochs.fif')

In [ ]:
channels = ['Fz', 'FC3', 'FC1', 'FCz', 'FC2', 'FC4', 'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'P1', 'Pz', 'P2', 'Oz', 'EoG1', 'EoG2', 'EoG3']

In [ ]:
nn_data_list = []
nn_labels = []
for event in epochs.event_id:
    nn_data_list.append(epochs[event].copy().crop(0, 4).pick(channels).get_data()[:,:,:-1])
    for i in range(len(epochs[event])):
            nn_labels.append(epochs.event_id[event])

In [ ]:
lb = LabelBinarizer().fit([1, 2, 3, 4])
nn_labels_OH = lb.transform(nn_labels)
nn_data = np.concatenate(nn_data_list, axis=0)
nn_data = np.transpose(nn_data, axes=[0,2,1])
print(nn_data.shape, nn_labels_OH.shape)

In [ ]:
X_train_nn, X_test_val_nn, y_train_nn, y_test_val_nn = train_test_split(nn_data,
                                                                        nn_labels_OH,
                                                                        stratify=nn_labels_OH,
                                                                        test_size=0.20,
                                                                        random_state=42)
X_test_nn, X_val_nn, y_test_nn, y_val_nn = train_test_split(X_test_val_nn,
                                                            y_test_val_nn,
                                                            stratify=y_test_val_nn,
                                                            test_size=0.50,
                                                            random_state=42)

In [ ]:
learning_rate = 1e-4
loss = tf.keras.losses.categorical_crossentropy
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model = CustomNet(inp_shape=(1000, 22))
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [ ]:
learning_rate = 1e-4
loss = tf.keras.losses.categorical_crossentropy
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

In [ ]:
modelPath = 'models/eegnet.h5'

checkpoint = ModelCheckpoint(
    modelPath,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='auto',
    save_freq='epoch'
)

earlystopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=4,
    restore_best_weights=True,
)

callbacksList = [checkpoint, earlystopping]  

In [ ]:
hist = model.fit(X_train_nn, y_train_nn, epochs=100, batch_size=10, validation_data=(X_val_nn,y_val_nn), shuffle=True, callbacks=callbacksList, verbose=1)